# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### --PENDING-- US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hopspitalization, ICU stats --PENDING--
- **Historic time series data:**
    - US State testing and hopspitalization, ICU stats --PENDING--

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-29-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-28-2020'

In [4]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,45001.0,Abbeville,South Carolina,US,2020-06-29 04:33:44,34.223334,-82.461707,103,0,0,103,"Abbeville, South Carolina, US",419.945366,0.000000
1,22001.0,Acadia,Louisiana,US,2020-06-29 04:33:44,30.295065,-92.414197,812,36,0,776,"Acadia, Louisiana, US",1308.727536,4.433498
2,51001.0,Accomack,Virginia,US,2020-06-29 04:33:44,37.767072,-75.632346,1039,14,0,1025,"Accomack, Virginia, US",3215.125634,1.347449
3,16001.0,Ada,Idaho,US,2020-06-29 04:33:44,43.452658,-116.241552,1841,23,0,1818,"Ada, Idaho, US",382.277761,1.249321
4,19001.0,Adair,Iowa,US,2020-06-29 04:33:44,41.330756,-94.471059,15,0,0,15,"Adair, Iowa, US",209.731544,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until day is over.**

In [6]:
jh_cases_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_cases_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273


In [7]:
jh_deaths_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_deaths_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,548,569,581,598,618,639,675,683,703,721
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,42,43,44,44,45,47,49,51,53,55
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,825,837,845,852,861,869,878,885,892,897


In [8]:
jh_recovered_global_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_recovered_global_hist.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7962,8292,8764,8841,9260,9869,10174,10306,10674,12604
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1114,1126,1134,1159,1195,1217,1250,1298,1346,1384
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,8196,8324,8422,8559,8674,8792,8920,9066,9202,9371


In [9]:
jh_cases_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_cases_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,200,222,222,222,225,226,231,247,247,247
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,30,30,30,30,30,30,30,30,30,30


In [10]:
jh_deaths_counties_hist = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_deaths_counties_hist.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [11]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
846,1051.0,Elmore,Alabama,US,2020-06-29 04:33:44,32.597854,-86.144153,774,13,0,761,"Elmore, Alabama, US",953.096332,1.679587
2652,1119.0,Sumter,Alabama,US,2020-06-29 04:33:44,32.591174,-88.199162,274,12,0,262,"Sumter, Alabama, US",2204.876479,4.379562
107,1001.0,Autauga,Alabama,US,2020-06-29 04:33:44,32.539527,-86.644082,497,11,0,486,"Autauga, Alabama, US",889.580984,2.213280


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [12]:
jh_countries_live = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [13]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,2548996,125803,685164
Brazil,1344143,57622,746018
Russia,633563,9060,398436


In [14]:
jh_countries_live['Active'] = jh_countries_live['Confirmed'] - jh_countries_live['Deaths'] - jh_countries_live['Recovered']

In [15]:
jh_countries_live['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_countries_live['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_countries_live['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [16]:
jh_countries_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,2548996,125803,685164,1738029,2020-06-29 04:33:44,38.286438,-91.591218
Brazil,1344143,57622,746018,540503,2020-06-29 04:33:44,-12.669522,-48.480493
Russia,633563,9060,398436,226067,2020-06-29 04:33:44,54.546312,62.120860
India,548318,16475,321723,210120,2020-06-29 04:33:44,23.356511,82.081064
United Kingdom,312640,43634,1364,267642,2020-06-29 04:33:44,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [17]:
jh_states_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [18]:
jh_states_live.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
New York,392539,31397,0
California,215296,5932,0
New Jersey,171182,14975,0


In [19]:
jh_states_live['Active'] = jh_states_live['Confirmed'] - jh_states_live['Deaths'] - jh_states_live['Recovered']

In [20]:
jh_states_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_states_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_states_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [21]:
jh_states_live.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
New York,392539,31397,0,361142,2020-06-29 04:33:44,42.671593,-75.579694
California,215296,5932,0,209364,2020-06-29 04:33:44,37.778248,-120.728666
New Jersey,171182,14975,0,156207,2020-06-29 04:33:44,40.286441,-74.622450
Texas,150152,2402,0,147750,2020-06-29 04:33:44,31.657778,-98.589862
Illinois,141723,6888,0,134835,2020-06-29 04:33:44,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [22]:
jh_counties_live = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,214750,22470,0
California,Los Angeles,97918,3305,0
Illinois,Cook,89755,4542,0
Arizona,Maricopa,44962,746,0
New York,Nassau,41754,2182,0


In [24]:
jh_counties_live['Active'] = jh_counties_live['Confirmed'] - jh_counties_live['Deaths'] - jh_counties_live['Recovered']


In [25]:
jh_counties_live['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_counties_live['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_counties_live['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_counties_live['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [26]:
jh_counties_live.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,214750,22470,0,192280,2020-06-29 04:33:44,40.767273,-73.971526,36061.0
California,Los Angeles,97918,3305,0,94613,2020-06-29 04:33:44,34.308284,-118.228241,6037.0
Illinois,Cook,89755,4542,0,85213,2020-06-29 04:33:44,41.841448,-87.816588,17031.0
Arizona,Maricopa,44962,746,0,44216,2020-06-29 04:33:44,33.348359,-112.491815,4013.0
New York,Nassau,41754,2182,0,39572,2020-06-29 04:33:44,40.740665,-73.589419,36059.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [27]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_cases_global_hist.iloc[:, 4:].columns

In [28]:
jh_cases_countries_hist_ts = pd.pivot_table(jh_cases_global_hist, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.transpose()
jh_cases_countries_hist_ts.index = pd.to_datetime(jh_cases_countries_hist_ts.index)
jh_cases_countries_hist_ts = jh_cases_countries_hist_ts.sort_index()

In [29]:
jh_cases_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,30451,2269,12685,855,212,65,55343,23247,7601,17522,...,310836,919,7427,4779,353,1557,10,1089,1531,561
2020-06-27,30616,2330,12968,855,259,65,57744,23909,7686,17580,...,311727,924,7682,5130,355,1815,10,1103,1531,567
2020-06-28,30967,2402,13273,855,267,69,59933,24645,7764,17654,...,312640,929,7948,5297,355,1990,10,1118,1557,567


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [30]:
jh_deaths_countries_hist_ts = pd.pivot_table(jh_deaths_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.transpose()
jh_deaths_countries_hist_ts.index = pd.to_datetime(jh_deaths_countries_hist_ts.index)
jh_deaths_countries_hist_ts = jh_deaths_countries_hist_ts.sort_index()

In [31]:
jh_deaths_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,683,51,885,52,10,3,1184,410,104,698,...,43498,26,20,41,0,3,1,293,21,6
2020-06-27,703,53,892,52,10,3,1207,421,104,700,...,43598,26,20,42,0,4,1,296,21,6
2020-06-28,721,55,897,52,11,3,1232,426,104,702,...,43634,27,22,44,0,4,1,302,22,6


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [32]:
jh_recovered_countries_hist_ts = pd.pivot_table(jh_recovered_global_hist, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.transpose()
jh_recovered_countries_hist_ts.index = pd.to_datetime(jh_recovered_countries_hist_ts.index)
jh_recovered_countries_hist_ts = jh_recovered_countries_hist_ts.sort_index()

In [33]:
jh_recovered_countries_hist_ts.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,10306,1298,9066,799,81,22,18416,12149,6960,16348,...,1363,818,5038,1327,330,446,8,402,1233,135
2020-06-27,10674,1346,9202,799,81,22,19143,12911,6993,16371,...,1364,818,5240,1327,330,446,8,417,1233,142
2020-06-28,12604,1384,9371,799,81,22,20134,13116,7007,16401,...,1364,818,5329,1649,330,447,8,430,1311,142


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [34]:
jh_cases_counties_hist.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
3147,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
52,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,1031,1054,1075,1097,1140,1182,1265,1326,1389,1408
51,84001093,US,USA,840,1093.0,Marion,Alabama,US,34.136974,-87.887042,...,165,168,168,168,171,179,183,184,188,189
50,84001091,US,USA,840,1091.0,Marengo,Alabama,US,32.247676,-87.787962,...,237,237,237,237,243,248,255,260,264,266
48,84001087,US,USA,840,1087.0,Macon,Alabama,US,32.387599,-85.692677,...,138,144,147,148,147,150,153,159,160,161


In [35]:
date_columns = jh_cases_counties_hist.iloc[:,11:].columns

In [36]:
jh_cases_states_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.transpose()
jh_cases_states_hist_ts.index = pd.to_datetime(jh_cases_states_hist_ts.index)
jh_cases_states_hist_ts = jh_cases_states_hist_ts.sort_index()

In [37]:
jh_cases_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,34183,833,0,66659,18740,206844,31780,46059,11017,49,...,39444,140173,20050,1198,81,60570,30855,2730,26747,1368
2020-06-27,35083,851,0,70124,19310,210584,32006,46206,11091,49,...,40172,145894,20628,1200,81,61247,31404,2782,27286,1392
2020-06-28,35441,880,0,73920,19818,215296,32290,46303,11226,49,...,40172,150152,21100,1202,81,61736,31752,2832,27743,1417


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [38]:
jh_cases_counties_hist_ts = pd.pivot_table(jh_cases_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.transpose()
jh_cases_counties_hist_ts.index = pd.to_datetime(jh_cases_counties_hist_ts.index)
jh_cases_counties_hist_ts = jh_cases_counties_hist_ts.sort_index()

In [39]:
jh_cases_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-26         482     500     309  150    181     347    592     228   
2020-06-27         492     539     314  158    185     354    597     237   
2020-06-28         497     559     314  159    186     353    599     237   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-26          545       65  ...      25      2       24        5   
2020-06-27          547       66  ...      31      2       24        6   
2020-06-28          547       67  ...      37      2       26        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-26             80   119   166          0       39      1  
2020-06-27             81   119   167          0       39      1  
2020-06-28             82   123   168          0       39      1  

[3 rows x 3254 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [40]:
jh_deaths_states_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.transpose()
jh_deaths_states_hist_ts.index = pd.to_datetime(jh_deaths_states_hist_ts.index)
jh_deaths_states_hist_ts = jh_deaths_states_hist_ts.sort_index()

In [41]:
jh_deaths_states_hist_ts.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,907,14,0,1536,249,5868,1673,4307,507,0,...,577,2354,166,56,6,1700,1304,92,766,20
2020-06-27,919,14,0,1580,259,5899,1674,4311,507,0,...,584,2388,167,56,6,1724,1310,93,777,20
2020-06-28,919,14,0,1594,264,5932,1676,4316,507,0,...,584,2402,167,56,6,1732,1310,93,777,20


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [42]:
jh_deaths_counties_hist_ts = pd.pivot_table(jh_deaths_counties_hist, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.transpose()
jh_deaths_counties_hist_ts.index = pd.to_datetime(jh_deaths_counties_hist_ts.index)
jh_deaths_counties_hist_ts = jh_deaths_counties_hist_ts.sort_index()

In [43]:
jh_deaths_counties_hist_ts.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-26          10       9       1    1      1      10     27       5   
2020-06-27          11       9       1    1      1      10     27       5   
2020-06-28          11       9       1    1      1      10     27       5   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-26           27        7  ...       0      0        0        0   
2020-06-27           27        7  ...       0      0        0        0   
2020-06-28           27        7  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-26              0     0     0         19        0      0  
2020-06-27              0     0     0         19        0      0  
2020-06-28              0     0     0         19        0      0  

[3 rows x 3254 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [44]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_countries_live.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [45]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_states_live.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [46]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_counties_live.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [47]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_cases_countries_hist_ts.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [48]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_deaths_countries_hist_ts.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_recovered_countries_hist_ts.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_cases_states_hist_ts.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_cases_counties_hist_ts.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_deaths_states_hist_ts.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_deaths_counties_hist_ts.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [54]:
jh_cases_countries_hist_pct = jh_cases_countries_hist_ts.pct_change()

In [55]:
jh_cases_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.009147,0.035128,0.019285,0.0,0.000000,0.000000,0.055016,0.033751,0.000790,0.002575,...,0.004463,0.013230,0.034833,0.047337,0.002841,0.126628,0.0,0.012082,0.022712,0.018149
2020-06-27,0.005419,0.026884,0.022310,0.0,0.221698,0.000000,0.043384,0.028477,0.011183,0.003310,...,0.002866,0.005441,0.034334,0.073446,0.005666,0.165703,0.0,0.012856,0.000000,0.010695
2020-06-28,0.011465,0.030901,0.023519,0.0,0.030888,0.061538,0.037909,0.030783,0.010148,0.004209,...,0.002929,0.005411,0.034626,0.032554,0.000000,0.096419,0.0,0.013599,0.016982,0.000000


In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_cases_countries_hist_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [57]:
jh_deaths_countries_hist_pct = jh_deaths_countries_hist_ts.pct_change()

In [58]:
jh_deaths_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.011852,0.040816,0.007973,0.0,0.0,0.0,0.029565,0.032746,0.0,0.000000,...,0.004248,0.000000,0.0,0.051282,NaN,0.000000,0.0,0.017361,0.166667,0.0
2020-06-27,0.029283,0.039216,0.007910,0.0,0.0,0.0,0.019426,0.026829,0.0,0.002865,...,0.002299,0.000000,0.0,0.024390,NaN,0.333333,0.0,0.010239,0.000000,0.0
2020-06-28,0.025605,0.037736,0.005605,0.0,0.1,0.0,0.020713,0.011876,0.0,0.002857,...,0.000826,0.038462,0.1,0.047619,NaN,0.000000,0.0,0.020270,0.047619,0.0


In [59]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_deaths_countries_hist_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [60]:
jh_recovered_countries_hist_pct = jh_recovered_countries_hist_ts.pct_change()

In [61]:
jh_recovered_countries_hist_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.012974,0.038400,0.016368,0.002509,0.0,0.0,0.245334,0.071813,0.000287,0.001716,...,0.001470,0.0,0.033012,0.000000,0.00304,0.000000,0.0,0.041451,0.008177,0.054688
2020-06-27,0.035707,0.036980,0.015001,0.000000,0.0,0.0,0.039477,0.062721,0.004741,0.001407,...,0.000734,0.0,0.040095,0.000000,0.00000,0.000000,0.0,0.037313,0.000000,0.051852
2020-06-28,0.180813,0.028232,0.018366,0.000000,0.0,0.0,0.051768,0.015878,0.002002,0.001833,...,0.000000,0.0,0.016985,0.242653,0.00000,0.002242,0.0,0.031175,0.063260,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_recovered_countries_hist_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [63]:
jh_cases_states_hist_pct = jh_cases_states_hist_ts.pct_change()

In [64]:
jh_cases_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,0.029422,0.024600,NaN,0.053381,0.037537,0.028502,0.010075,0.001413,0.003370,0.0,...,0.037072,0.041729,0.034892,0.005877,0.0125,0.010409,0.016070,0.013363,0.019827,0.031674
2020-06-27,0.026329,0.021609,NaN,0.051981,0.030416,0.018081,0.007111,0.003192,0.006717,0.0,...,0.018457,0.040814,0.028828,0.001669,0.0000,0.011177,0.017793,0.019048,0.020152,0.017544
2020-06-28,0.010204,0.034078,NaN,0.054133,0.026308,0.022376,0.008873,0.002099,0.012172,0.0,...,0.000000,0.029186,0.022882,0.001667,0.0000,0.007984,0.011081,0.017973,0.016749,0.017960


In [65]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_cases_states_hist_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [66]:
jh_cases_counties_hist_pct = jh_cases_counties_hist_ts.pct_change()

In [67]:
jh_cases_counties_hist_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-26      0.019027  0.082251  0.019802  0.027397  0.046243  0.000000   
2020-06-27      0.020747  0.078000  0.016181  0.053333  0.022099  0.020173   
2020-06-28      0.010163  0.037106  0.000000  0.006329  0.005405 -0.002825   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-06-26      0.010239  0.013333  0.018692  0.048387  ...  0.136364    0.0   
2020-06-27      0.008446  0.039474  0.003670  0.015385  ...  0.240000    0.0   
2020-06-28      0.003350  0.000000  0.000000  0.015152  ...  0.193548    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-06-26      0.090909      0.0   0.095890  0.008475  0.024691        NaN   
2020-06-27      0.000000      0.2   0.012500  0.000000  0.006024        NaN   
2020-06-28      0.083333      0.0   0.012346  0.033613  0.005988        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-06-26          0.0    0.0  
2020-06-27          0.0    0.0  
2020-06-28          0.0    0.0  

[3 rows x 3254 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_cases_counties_hist_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_deaths_states_hist_pct = jh_deaths_states_hist_ts.pct_change()

In [70]:
jh_deaths_states_hist_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,0.012277,0.166667,NaN,0.027425,0.037500,0.010679,0.002397,0.002094,0.0,NaN,...,0.017637,0.015969,0.012195,0.0,0.0,0.014925,0.003077,0.00000,0.00000,0.0
2020-06-27,0.013230,0.000000,NaN,0.028646,0.040161,0.005283,0.000598,0.000929,0.0,NaN,...,0.012132,0.014444,0.006024,0.0,0.0,0.014118,0.004601,0.01087,0.01436,0.0
2020-06-28,0.000000,0.000000,NaN,0.008861,0.019305,0.005594,0.001195,0.001160,0.0,NaN,...,0.000000,0.005863,0.000000,0.0,0.0,0.004640,0.000000,0.00000,0.00000,0.0


In [71]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_deaths_states_hist_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [72]:
jh_deaths_counties_hist_pct = jh_deaths_counties_hist_ts.pct_change()

In [73]:
jh_deaths_counties_hist_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-06-26         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-06-27         0.1     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-06-28         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-06-26          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-27          0.0      0.0  ...     NaN    NaN      NaN      NaN   
2020-06-28          0.0      0.0  ...     NaN    NaN      NaN      NaN   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-06-26            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-27            NaN   NaN   NaN        0.0      NaN    NaN  
2020-06-28            NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3254 columns]

In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_deaths_counties_hist_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [75]:
jh_cases_countries_hist_pct_ewma = jh_cases_countries_hist_pct.ewm(span=3).mean()

In [76]:
jh_cases_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.011330,0.034104,0.017014,2.899013e-06,0.027011,0.375000,0.053643,0.032640,0.002642,0.002210,...,0.003819,0.011210,0.033829,0.044719,0.002619,0.110252,0.003472,0.026945,0.015835,0.024037
2020-06-27,0.008375,0.030494,0.019662,1.449506e-06,0.124355,0.187500,0.048513,0.030558,0.006912,0.002760,...,0.003343,0.008325,0.034082,0.059083,0.004142,0.137978,0.001736,0.019900,0.007918,0.017366
2020-06-28,0.009920,0.030698,0.021591,7.247532e-07,0.077621,0.124519,0.043211,0.030671,0.008530,0.003485,...,0.003136,0.006868,0.034354,0.045818,0.002071,0.117198,0.000868,0.016750,0.012450,0.008683


In [77]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_cases_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [78]:
jh_deaths_countries_hist_pct_ewma = jh_deaths_countries_hist_pct.ewm(span=3).mean()

In [79]:
jh_deaths_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.028121,0.038965,0.009042,0.000010,0.004907,1.067063e-22,0.030563,0.032580,0.001826,0.002135,...,0.003924,0.005347,0.013158,0.045961,NaN,1.455192e-11,0.0,0.029302,0.123119,0.007812
2020-06-27,0.028702,0.039090,0.008476,0.000005,0.002453,5.335315e-23,0.024994,0.029705,0.000913,0.002500,...,0.003111,0.002673,0.006579,0.035176,NaN,1.666667e-01,0.0,0.019770,0.061560,0.003906
2020-06-28,0.027153,0.038413,0.007041,0.000002,0.051227,2.667657e-23,0.022853,0.020791,0.000457,0.002679,...,0.001969,0.020567,0.053290,0.041397,NaN,8.333333e-02,0.0,0.020020,0.054589,0.001953


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_deaths_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [81]:
jh_recovered_countries_hist_pct_ewma = jh_recovered_countries_hist_pct.ewm(span=3).mean()

In [82]:
jh_recovered_countries_hist_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-06-26,0.027714,0.031244,0.015297,0.001499,0.015756,0.000049,0.146938,0.065260,0.001418,0.001814,...,0.005611,0.000978,0.032737,0.018501,0.001845,0.002755,0.000020,0.039180,0.006300,0.153082
2020-06-27,0.031711,0.034112,0.015149,0.000750,0.007878,0.000024,0.093207,0.063991,0.003080,0.001611,...,0.003173,0.000489,0.036416,0.009250,0.000923,0.001378,0.000010,0.038247,0.003150,0.102467
2020-06-28,0.106262,0.031172,0.016757,0.000375,0.003939,0.000012,0.072488,0.039934,0.002541,0.001722,...,0.001586,0.000245,0.026701,0.125951,0.000461,0.001810,0.000005,0.034711,0.033205,0.051234


In [83]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_recovered_countries_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [84]:
jh_cases_states_hist_pct_ewma = jh_cases_states_hist_pct.ewm(span=3).mean()

In [85]:
jh_cases_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,0.029886,0.025699,NaN,0.050755,0.038246,0.027812,0.009457,0.001420,0.004795,6.713710e-32,...,0.029448,0.043851,0.031697,0.006951,0.020064,0.009251,0.016121,0.016185,0.017922,0.029755
2020-06-27,0.028107,0.023654,NaN,0.051368,0.034331,0.022947,0.008284,0.002306,0.005756,3.356855e-32,...,0.023952,0.042332,0.030263,0.004310,0.010032,0.010214,0.016957,0.017616,0.019037,0.023649
2020-06-28,0.019156,0.028866,NaN,0.052750,0.030319,0.022661,0.008579,0.002203,0.008964,1.678427e-32,...,0.011976,0.035759,0.026572,0.002989,0.005016,0.009099,0.014019,0.017795,0.017893,0.020805


In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_cases_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [87]:
jh_cases_counties_hist_pct_ewma = jh_cases_counties_hist_pct.ewm(span=3).mean()

In [88]:
jh_cases_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-06-26      0.022314  0.055553  0.029043  0.037756  0.047149  0.014309   
2020-06-27      0.021530  0.066777  0.022612  0.045545  0.034624  0.017241   
2020-06-28      0.015846  0.051941  0.011306  0.025937  0.020015  0.007208   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-06-26      0.008163  0.025241  0.015939  0.051292  ...  0.175423   
2020-06-27      0.008305  0.032357  0.009804  0.033338  ...  0.207711   
2020-06-28      0.005827  0.016179  0.004902  0.024245  ...  0.200630   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-06-26      0.031250  0.084929  0.008464   0.099719  0.016928  0.030318   
2020-06-27      0.015625  0.042464  0.104232   0.056110  0.008464  0.018171   
2020-06-28      0.007813  0.062899  0.052116   0.034228  0.021039  0.012080   

Province_State                                  
Admin2         Unassigned      Washakie Weston  
2020-06-26            NaN  6.971132e-09    0.0  
2020-06-27            NaN  3.485566e-09    0.0  
2020-06-28            NaN  1.742783e-09    0.0  

[3 rows x 3254 columns]

In [89]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_cases_counties_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [90]:
jh_deaths_states_hist_pct_ewma = jh_deaths_states_hist_pct.ewm(span=3).mean()

In [91]:
jh_deaths_states_hist_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-06-26,0.013524,0.083336,NaN,0.027354,0.024090,0.012304,0.002308,0.002283,0.011255,NaN,...,0.018903,0.016384,0.010071,0.000036,4.618601e-15,0.012047,0.004496,0.002285,0.004848,0.000434
2020-06-27,0.013377,0.041668,NaN,0.028000,0.032126,0.008794,0.001453,0.001606,0.005628,NaN,...,0.015517,0.015414,0.008048,0.000018,2.309300e-15,0.013082,0.004548,0.006577,0.009604,0.000217
2020-06-28,0.006689,0.020834,NaN,0.018430,0.025715,0.007194,0.001324,0.001383,0.002814,NaN,...,0.007759,0.010638,0.004024,0.000009,1.154650e-15,0.008861,0.002274,0.003289,0.004802,0.000109


In [92]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_deaths_states_hist_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [93]:
jh_deaths_counties_hist_pct_ewma = jh_deaths_counties_hist_pct.ewm(span=3).mean()

In [94]:
jh_deaths_counties_hist_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga       Baldwin Barbour Bibb Blount   Bullock   
2020-06-26      0.032693  1.830930e-12     0.0  0.0    0.0  0.014377   
2020-06-27      0.066347  9.154648e-13     0.0  0.0    0.0  0.007189   
2020-06-28      0.033173  4.577324e-13     0.0  0.0    0.0  0.003594   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2020-06-26      0.002561  0.015788  0.000021  0.011206  ...     NaN    NaN   
2020-06-27      0.001280  0.007894  0.000011  0.005603  ...     NaN    NaN   
2020-06-28      0.000640  0.003947  0.000005  0.002801  ...     NaN    NaN   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2020-06-26          NaN      NaN        NaN   NaN   NaN   0.000460      NaN   
2020-06-27          NaN      NaN        NaN   NaN   NaN   0.000230      NaN   
2020-06-28          NaN      NaN        NaN   NaN   NaN   0.000115      NaN   

Province_State         
Admin2         Weston  
2020-06-26        NaN  
2020-06-27        NaN  
2020-06-28        NaN  

[3 rows x 3254 columns]

In [95]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_deaths_counties_hist_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

# TODO - CONTINUE FROM HERE

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/states | [CSV](https://covidtracking.com/api/states.csv)  
States daily 4 pm ET - /api/states/daily | [CSV](http://covidtracking.com/api/states/daily.csv)  
States info - /api/states/info | [CSV](https://covidtracking.com/api/states/info.csv)  
US current - /api/us | [CSV](https://covidtracking.com/api/us.csv)  
US daily - /api/us/daily | [CSV](https://covidtracking.com/api/us/daily.csv)  
Counties- /api/counties | [CSV](https://covidtracking.com/api/counties.csv)  